In [5]:
%%writefile serial.cpp
#include <iostream>
#include <vector>
#include <fstream>
#include <limits>
#include <chrono>
using namespace std;

struct Edge {
    int from, to;
    long long capacity, flow;
    Edge(int u, int v, long long cap) : from(u), to(v), capacity(cap), flow(0) {}
};

class MaxFlow {
    int V;
    vector<Edge> edges;
    vector<vector<int>> adj;
    vector<int> height, excess;

public:
    MaxFlow(int vertices) : V(vertices), adj(vertices), height(vertices, 0), excess(vertices, 0) {}

    void addEdge(int u, int v, long long capacity) {
        edges.emplace_back(u, v, capacity);
        edges.emplace_back(v, u, 0);  // Reverse edge with 0 capacity
        adj[u].push_back(edges.size() - 2);
        adj[v].push_back(edges.size() - 1);
    }

    void push(int e) {
        Edge &edge = edges[e];
        long long pushFlow = min((long long)excess[edge.from], (long long)(edge.capacity - edge.flow));
        if (pushFlow > 0 && height[edge.from] == height[edge.to] + 1) {
            edge.flow += pushFlow;
            edges[e ^ 1].flow -= pushFlow;  // Reverse edge update
            excess[edge.from] -= pushFlow;
            excess[edge.to] += pushFlow;
        }
    }

    void relabel(int u) {
        int minHeight = numeric_limits<int>::max();
        for (int e : adj[u]) {
            if (edges[e].capacity > edges[e].flow) {
                minHeight = min(minHeight, height[edges[e].to]);
            }
        }
        if (minHeight < numeric_limits<int>::max()) {
            height[u] = minHeight + 1;
        }
    }

    void discharge(int u) {
        while (excess[u] > 0) {
            bool pushed = false;
            for (int e : adj[u]) {
                if (edges[e].capacity > edges[e].flow && height[u] == height[edges[e].to] + 1) {
                    push(e);
                    pushed = true;
                }
            }
            if (!pushed) {
                relabel(u);
            }
        }
    }

    long long maxFlow(int source, int sink) {
        height[source] = V;
        excess[source] = 0;

        // Initialize preflow
        for (int e : adj[source]) {
            Edge &edge = edges[e];
            if (edge.capacity > 0) {
                long long pushFlow = edge.capacity;
                edge.flow = pushFlow;
                edges[e ^ 1].flow = -pushFlow;
                excess[edge.to] += pushFlow;
                excess[source] -= pushFlow;
            }
        }

        vector<int> activeNodes;
        for (int i = 1; i < V - 1; ++i) {
            if (excess[i] > 0) {
                activeNodes.push_back(i);
            }
        }

        for (int u : activeNodes) {
            discharge(u);
        }

        long long totalFlow = 0;
        for (int e : adj[source]) {
            totalFlow += edges[e].flow;
        }
        return totalFlow;
    }
};

int main() {
    ifstream infile("/kaggle/input/hpc-cuda/input.txt");
    if (!infile) {
        cerr << "Error opening file." << endl;
        return 1;
    }

    int V, source, sink;
    infile >> V >> source >> sink;
    MaxFlow maxFlow(V);

    int u, v;
    long long capacity;
    while (infile >> u >> v >> capacity) {
        maxFlow.addEdge(u, v, capacity);
    }
    infile.close();
    auto start = chrono::high_resolution_clock::now();
    long long result = maxFlow.maxFlow(source, sink);
    auto end = chrono::high_resolution_clock::now();
    double duration = chrono::duration<double>(end - start).count();
    cout << "Max Flow: " << result << endl;
    cout << "Execution Time: " << duration << " s" << endl;

    return 0;
}


Overwriting serial.cpp


In [6]:
!g++ -o maxflow serial.cpp && ./maxflow


Max Flow: 42406
Execution Time: 0.213767 s


In [9]:
%%writefile push_relabel.cu
#include <iostream>
#include <vector>
#include <fstream>
#include <cuda_runtime.h>
using namespace std;

struct Edge {
    int from, to;
    long long capacity, flow;
    Edge(int u, int v, long long cap) : from(u), to(v), capacity(cap), flow(0) {}
};

__device__ void push(int e, Edge *edges, int *height, long long *excess) {
    Edge &edge = edges[e];
    long long pushFlow = min(excess[edge.from], edge.capacity - edge.flow);
    if (pushFlow > 0 && height[edge.from] == height[edge.to] + 1) {
        edge.flow += pushFlow;
        edges[e ^ 1].flow -= pushFlow;
        excess[edge.from] -= pushFlow;
        excess[edge.to] += pushFlow;
    }
}

__device__ void relabel(int u, Edge *edges, int *height, int *adj, int *adjSize) {
    int minHeight = INT_MAX;
    for (int i = 0; i < adjSize[u]; ++i) {
        int e = adj[u * adjSize[u] + i];
        if (edges[e].capacity > edges[e].flow)
            minHeight = min(minHeight, height[edges[e].to]);
    }
    if (minHeight < INT_MAX)
        height[u] = minHeight + 1;
}

__global__ void dischargeKernel(int *activeNodes, Edge *edges, int *height, long long *excess, int *adj, int *adjSize, int numActive) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < numActive) {
        int u = activeNodes[idx];
        while (excess[u] > 0) {
            bool workDone = false;
            for (int i = 0; i < adjSize[u]; ++i) {
                int e = adj[u * adjSize[u] + i];
                if (edges[e].capacity > edges[e].flow && height[u] == height[edges[e].to] + 1) {
                    push(e, edges, height, excess);
                    workDone = true;
                }
            }
            if (!workDone) {
                relabel(u, edges, height, adj, adjSize);
            }
        }
    }
}

class MaxFlow {
    int V;
    vector<Edge> edges;
    vector<vector<int>> adj;
    vector<int> height;
    vector<long long> excess;

public:
    MaxFlow(int vertices) : V(vertices), adj(vertices), height(vertices, 0), excess(vertices, 0) {}

    void addEdge(int u, int v, long long capacity) {
        edges.emplace_back(u, v, capacity);
        edges.emplace_back(v, u, 0);
        adj[u].push_back(edges.size() - 2);
        adj[v].push_back(edges.size() - 1);
    }

    long long maxFlow(int source, int sink) {
        height[source] = V;
        excess[source] = 0;

        for (int e : adj[source]) {
            Edge &edge = edges[e];
            if (edge.capacity > 0) {
                long long pushFlow = edge.capacity;
                edge.flow = pushFlow;
                edges[e ^ 1].flow = -pushFlow;
                excess[edge.to] += pushFlow;
                excess[source] -= pushFlow;
            }
        }

        Edge *d_edges;
        int *d_height, *d_adj, *d_adjSize;
        long long *d_excess;
        int *d_activeNodes;
        int numActive = V - 2;
        int *activeNodes = new int[numActive];
        int *adjSize = new int[V];
        for (int i = 1, j = 0; i < V - 1; ++i) activeNodes[j++] = i;
        for (int i = 0; i < V; ++i) adjSize[i] = adj[i].size();

        double memStart = clock();
        cudaMalloc(&d_edges, edges.size() * sizeof(Edge));
        cudaMalloc(&d_height, V * sizeof(int));
        cudaMalloc(&d_excess, V * sizeof(long long));
        cudaMalloc(&d_adj, V * sizeof(int) * V);
        cudaMalloc(&d_adjSize, V * sizeof(int));
        cudaMalloc(&d_activeNodes, numActive * sizeof(int));
        
        cudaMemcpy(d_edges, edges.data(), edges.size() * sizeof(Edge), cudaMemcpyHostToDevice);
        cudaMemcpy(d_height, height.data(), V * sizeof(int), cudaMemcpyHostToDevice);
        cudaMemcpy(d_excess, excess.data(), V * sizeof(long long), cudaMemcpyHostToDevice);
        cudaMemcpy(d_activeNodes, activeNodes, numActive * sizeof(int), cudaMemcpyHostToDevice);
        cudaMemcpy(d_adjSize, adjSize, V * sizeof(int), cudaMemcpyHostToDevice);
        double memEnd = clock();

        int blockSize = 512;
        int gridSize = (numActive + blockSize - 1) / blockSize;
        double execStart = clock();
        dischargeKernel<<<gridSize, blockSize>>>(d_activeNodes, d_edges, d_height, d_excess, d_adj, d_adjSize, numActive);
        cudaDeviceSynchronize();
        double execEnd = clock();

        cudaMemcpy(edges.data(), d_edges, edges.size() * sizeof(Edge), cudaMemcpyDeviceToHost);
        cudaFree(d_edges);
        cudaFree(d_height);
        cudaFree(d_excess);
        cudaFree(d_adj);
        cudaFree(d_adjSize);
        cudaFree(d_activeNodes);
        delete[] activeNodes;
        delete[] adjSize;

        long long totalFlow = 0;
        for (int e : adj[source]) {
            totalFlow += edges[e].flow;
        }
        cout << "Memory Copy Time: " << (memEnd - memStart) / CLOCKS_PER_SEC << "s, Kernel Execution Time: " << (execEnd - execStart) / CLOCKS_PER_SEC << "s" << endl;
        return totalFlow;
    }
};

int main() {
    ifstream infile("/kaggle/input/hpc-cuda/input.txt");
    if (!infile) {
        cerr << "Error opening file." << endl;
        return 1;
    }
    int V, source, sink;
    infile >> V >> source >> sink;
    MaxFlow maxFlow(V);
    int u, v;
    long long capacity;
    while (infile >> u >> v >> capacity) {
        maxFlow.addEdge(u, v, capacity);
    }
    infile.close();

    long long result = maxFlow.maxFlow(source, sink);
    cout << "Max Flow: " << result << endl;
    return 0;
}



Overwriting push_relabel.cu


In [10]:
!nvcc -o push_relabel push_relabel.cu && ./push_relabel

Memory Copy Time: 0.097116s, Kernel Execution Time: 0.044733s
Max Flow: 42406
